In [215]:
import awswrangler as wr
import pandas as pd
pd.set_option('display.max_rows', 300)


In [216]:
my_filter = lambda x: x["CoType"] == "I"

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket = "tmp_part"


In [217]:
wr.s3.read_parquet_metadata(f"s3://{bucket}/")


({'CaseID': 'bigint',
  'iDPTID': 'double',
  'PartNo': 'string',
  'PartGroup': 'double',
  'PartRefCode': 'string',
  'CatRefCode': 'string',
  'Description': 'string',
  'AddDescription': 'string',
  'Comments': 'string',
  'DamageCond': 'string',
  'Quantity': 'double',
  'fFDB': 'double',
  'siNETTDB': 'bigint',
  'fVAL': 'double',
  'siPINS': 'double'},
 None)

In [218]:
my_filter = lambda x: x["CoType"] == "I"

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"
bucket = "tmp_part"

partDf = wr.s3.read_parquet(f"s3://{bucket}/",  dataset=True, partition_filter=my_filter, columns=["CaseID", "Description", "fVAL", "DamageCond"])
partDf

,CaseID,Description,fVAL,DamageCond,CoType
0,10357852,Sealant,80.0,,I
1,10357852,Windscreen Glass - Front,230.0,,I
2,10357855,Air Press Set,298.3,Broken,I
3,10357855,Door Rear RH,1200.0,Bent/dented/cut,I
4,10357855,Door Outer Release Handle,377.8,Deeply cut,I
...,...,...,...,...,...
3396343,10201018,REAR BUMPER,3615.0,,I
3396344,10201018,TRIM QUARTER PANEL RH,1008.0,,I
3396345,10201018,VENT TRIM,367.5,,I
3396346,10201018,TAILAMP RH,1271.0,,I


In [219]:
partDf["DamageCond"] = partDf["DamageCond"].str.lower()
partDf["Description"] = partDf["Description"].str.lower()
partDf

,CaseID,Description,fVAL,DamageCond,CoType
0,10357852,sealant,80.0,,I
1,10357852,windscreen glass - front,230.0,,I
2,10357855,air press set,298.3,broken,I
3,10357855,door rear rh,1200.0,bent/dented/cut,I
4,10357855,door outer release handle,377.8,deeply cut,I
...,...,...,...,...,...
3396343,10201018,rear bumper,3615.0,,I
3396344,10201018,trim quarter panel rh,1008.0,,I
3396345,10201018,vent trim,367.5,,I
3396346,10201018,tailamp rh,1271.0,,I


In [220]:
rawPartsTotal = len(partDf)

In [221]:
costThreshold = 100

In [222]:
removedByLowCost = len(partDf[(partDf["fVAL"] < costThreshold)])


In [223]:
nullCostParts = len(partDf[partDf["fVAL"].isnull()]) / len(partDf)
nullCostParts


0.1984640560979028

In [224]:
DmgCondFilter = [
    "no visible damage", 
    "to check", 
    "useable",
    "not necessary",
    "not damaged",
    "slightly bent",
    "slightly cut",
    "slightly scratched",
    "slightly dented",
    "visually intact"
]

In [225]:
removedByCond = len(partDf[partDf["DamageCond"].isin(DmgCondFilter)])
removedByCond

236172

In [226]:
removedByDash  = len(partDf[partDf["Description"] == "-"])


In [227]:
removedByNoCost  = len(partDf[partDf["fVAL"].isna()])


In [228]:
filterPartDf = partDf[(partDf["fVAL"] > costThreshold)]
filterPartDf = filterPartDf[~filterPartDf["DamageCond"].isin(DmgCondFilter)]
filterPartDf = filterPartDf[filterPartDf["Description"] != "-"]


In [229]:
import plotly.express as ex
import plotly.graph_objects as go


fig = go.Figure(data=[go.Pie(labels=["Part Data Used", 
                                     "Removed by No Part Description", \
                                     "Removed by damage conditions", \
                                        "Removed by cost < RM100", \
                                     "Removed by no part cost" \
                                     ], \
                             values=[len(filterPartDf), removedByDash, removedByCond, removedByLowCost, removedByNoCost])])
fig.update_layout(title="Partlist Filtering")

fig.show()


In [230]:
# filteredPartDf["DamageCond"].value_counts().head(n=100).reset_index()
# filterPartDf["Description"].value_counts().reset_index().head(300)

,index,Description
0,windscreen glass - front,83838
1,bumper front cover,62163
2,headlamp assy rh,44528
3,headlamp assy lh,43488
4,tint film,41721
5,bonnet,39354
6,bumper rear cover,31603
7,radiator,30187
8,air conditioning condenser,28684
9,fender front rh,20968


,CaseID,Description,fVAL,DamageCond,CoType
